In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from sklearn.metrics import classification_report

from keras.models import Sequential
from keras.layers import LSTM,Bidirectional,Dense,Embedding,Dropout

In [2]:
train_data = pd.read_csv('train.txt',header =None, sep =';', names = ['Input','Sentiment'], encoding='utf-8')
train_data.head()

,Input,Sentiment
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [3]:
test_data = pd.read_csv('test.txt',header=None, sep=';', names= ['Input','Sentiment'], encoding='utf-8')
test_data.head()

,Input,Sentiment
0,im feeling rather rotten so im not very ambiti...,sadness
1,im updating my blog because i feel shitty,sadness
2,i never make her separate from me because i do...,sadness
3,i left with my bouquet of red and yellow tulip...,joy
4,i was feeling a little vain when i did this one,sadness


In [4]:
val_data = pd.read_csv('val.txt', header=None,sep=';', names=['Input','Sentiment'], encoding='utf-8')
val_data.head()

,Input,Sentiment
0,im feeling quite sad and sorry for myself but ...,sadness
1,i feel like i am still looking at a blank canv...,sadness
2,i feel like a faithful servant,love
3,i am just feeling cranky and blue,anger
4,i can have for a treat or if i am feeling festive,joy


In [5]:
print(train_data.isnull().sum())
print(test_data.isnull().sum())
print(val_data.isnull().sum())

Input        0
Sentiment    0
dtype: int64
Input        0
Sentiment    0
dtype: int64
Input        0
Sentiment    0
dtype: int64


In [6]:
train_data.Sentiment.value_counts()

joy         5362
sadness     4666
anger       2159
fear        1937
love        1304
surprise     572
Name: Sentiment, dtype: int64

In [7]:
train_data.Input

0                                  i didnt feel humiliated
1        i can go from feeling so hopeless to so damned...
2         im grabbing a minute to post i feel greedy wrong
3        i am ever feeling nostalgic about the fireplac...
4                                     i am feeling grouchy
                               ...                        
15995    i just had a very brief time in the beanbag an...
15996    i am now turning and i feel pathetic that i am...
15997                       i feel strong and good overall
15998    i feel like this was such a rude comment and i...
15999    i know a lot but i feel so stupid because i ca...
Name: Input, Length: 16000, dtype: object

In [8]:
tokenizer=Tokenizer(16000,lower=True,oov_token='UNK')
tokenizer.fit_on_texts(train_data.Input)

In [9]:
X_train=tokenizer.texts_to_sequences(train_data.Input)
X_train_pad=pad_sequences(X_train,maxlen=80,padding='post')

In [10]:
train_data['Sentiment']= train_data.Sentiment.replace({'joy':0,'anger':1,'love':2,'sadness':3,'fear':4,'surprise':5})
y_train = train_data['Sentiment'].values
y_train

array([3, 3, 1, ..., 0, 1, 3])

In [11]:
y_train_f=to_categorical(y_train)
y_train_f

array([[0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       ...,
       [1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.]], dtype=float32)

In [12]:
x_val_data=val_data['Input']
y_val_data=val_data.Sentiment.replace({'joy':0,'anger':1,'love':2,'sadness':3,'fear':4,'surprise':5})

X_val_=tokenizer.texts_to_sequences(x_val_data)
X_val_pad=pad_sequences(X_val_,maxlen=80,padding='post')


In [13]:
Y_val_= to_categorical(y_val_data)
Y_val_

array([[0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       ...,
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.]], dtype=float32)

In [14]:
model = Sequential()
model.add(Embedding(16000,64, input_length=80))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(80, return_sequences=True)))
model.add(Bidirectional(LSTM(160)))
model.add(Dense(6,activation='softmax'))
print(model.summary())


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 80, 64)            1024000   
_________________________________________________________________
dropout (Dropout)            (None, 80, 64)            0         
_________________________________________________________________
bidirectional (Bidirectional (None, 80, 160)           92800     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 320)               410880    
_________________________________________________________________
dense (Dense)                (None, 6)                 1926      
Total params: 1,529,606
Trainable params: 1,529,606
Non-trainable params: 0
_________________________________________________________________
None


In [15]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [17]:
Emoji = model.fit(x=X_train_pad,y=y_train_f, epochs=16, validation_data=(X_val_pad,Y_val_))

Epoch 1/16
500/500 [==============================] - 12s 24ms/step - loss: 0.0343 - accuracy: 0.9876 - val_loss: 0.3087 - val_accuracy: 0.9230
Epoch 2/16
500/500 [==============================] - 12s 25ms/step - loss: 0.0321 - accuracy: 0.9874 - val_loss: 0.2821 - val_accuracy: 0.9285
Epoch 3/16
500/500 [==============================] - 12s 25ms/step - loss: 0.0313 - accuracy: 0.9885 - val_loss: 0.2984 - val_accuracy: 0.9255
Epoch 4/16
500/500 [==============================] - 12s 25ms/step - loss: 0.0312 - accuracy: 0.9886 - val_loss: 0.2747 - val_accuracy: 0.9275
Epoch 5/16
500/500 [==============================] - 12s 24ms/step - loss: 0.0293 - accuracy: 0.9898 - val_loss: 0.2864 - val_accuracy: 0.9250
Epoch 6/16
500/500 [==============================] - 13s 25ms/step - loss: 0.0266 - accuracy: 0.9903 - val_loss: 0.2380 - val_accuracy: 0.9300
Epoch 7/16
500/500 [==============================] - 12s 25ms/step - loss: 0.0227 - accuracy: 0.9914 - val_loss: 0.2781 - val_accuracy:

In [18]:
X_test=test_data['Input']
Y_test=test_data.Sentiment.replace({'joy':0,'anger':1,'love':2,'sadness':3,'fear':4,'surprise':5})


In [21]:
X_test_f=tokenizer.texts_to_sequences(X_test)
X_test_pad=pad_sequences(X_test_f,maxlen=80,padding='post')
X_test_pad

array([[   17,     8,   203, ...,     0,     0,     0],
       [   17, 12800,    11, ...,     0,     0,     0],
       [    2,   145,    80, ...,     0,     0,     0],
       ...,
       [    2,     3,     9, ...,     0,     0,     0],
       [   17,     8,    38, ...,     0,     0,     0],
       [    2,     3,    36, ...,     0,     0,     0]], dtype=int32)

In [20]:
Y_test_f=to_categorical(Y_test)
Y_test_f

array([[0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       ...,
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.]], dtype=float32)

In [22]:
model.evaluate(X_test_pad,Y_test_f)

63/63 [==============================] - 1s 11ms/step - loss: 0.3186 - accuracy: 0.9265


[0.3185918927192688, 0.9265000224113464]

In [25]:
Y_pred=model.predict_classes(X_test_pad)
Y_pred

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array([3, 3, 3, ..., 0, 0, 4])

In [27]:
print(classification_report(Y_test,Y_pred))

              precision    recall  f1-score   support

           0       0.93      0.95      0.94       695
           1       0.96      0.91      0.94       275
           2       0.83      0.81      0.82       159
           3       0.96      0.98      0.97       581
           4       0.90      0.87      0.88       224
           5       0.77      0.76      0.76        66

    accuracy                           0.93      2000
   macro avg       0.89      0.88      0.89      2000
weighted avg       0.93      0.93      0.93      2000



DEFINE A FUNCUTION THAT PREDICT YOUR ENTER WORD OR SENTENCSE

In [28]:
def get_key(value):
    dictionary={'joy':0,'anger':1,'love':2,'sadness':3,'fear':4,'surprise':5}
    for key,val in dictionary.items():
          if (val==value):
            return key

In [29]:
def predict(sentence):
  sentence_lst=[]
  sentence_lst.append(sentence)
  sentence_seq=tokenizer.texts_to_sequences(sentence_lst)
  sentence_padded=pad_sequences(sentence_seq,maxlen=80,padding='post')
  ans=get_key(model.predict_classes(sentence_padded))
  print("The emotion predicted is",ans)


In [30]:
predict(str(input('Enter a sentence : ')))

Enter a sentence : I REALLY LOVE YOU 
The emotion predicted is joy


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [33]:
predict(str(input('Enter a sentence : ')))


Enter a sentence : at the moment that i knowe i fucked up
The emotion predicted is anger


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [ ]:
predict(str(input('Enter a sentence : ')))

In [ ]:
predict(str(input('Enter a sentence : ')))